# Исследование надежности заемщиков


## 1. Первичное рассмотрение данных

Загрузим данные, выведем первые строки и основную информацию о датафрейме.

In [1]:
import pandas as pd

data = pd.read_csv('datasets/data.csv')

In [2]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

Выведем количество пропущенных значений для каждого столбца.

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Можем видеть, что пропуски присутствуют в двух столбцах - `days_employed` и `total_income`. 

Заполним пропуски в `total_income` и `days_employed` медианным значением по каждому типу `income_type`.

In [ ]:
data['income_type'].unique()

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

In [ ]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

Убедимся, что все пропуски заполнены.

In [ ]:
data.isna().sum()

### Обработка аномальных значений

В данных присутствуют артефакты - (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. В данном случае это отрицательное количество дней трудового стажа в столбце `days_employed`. Обработаем значения методом `abs()`

In [6]:
#data.loc[data['days_employed'] < 0, 'days_employed']

0       -8437.673028
1       -4024.803754
2       -5623.422610
3       -4124.747207
5        -926.185831
            ...     
21519   -2351.431934
21520   -4529.316663
21522   -2113.346888
21523   -3112.481705
21524   -1984.507589
Name: days_employed, Length: 15906, dtype: float64

In [7]:
data['days_employed'].describe()

count     19351.000000
mean      63046.497661
std      140827.311974
min      -18388.949901
25%       -2747.423625
50%       -1203.369529
75%        -291.095954
max      401755.400475
Name: days_employed, dtype: float64

In [8]:
data['days_employed'] = data['days_employed'].abs()

Для каждого типа занятости выведем медианное значение трудового стажа в днях.

In [28]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.137274
компаньон            1555.947387
пенсионер          365286.622650
предприниматель       520.848083
сотрудник            1572.328285
студент               578.751554
Name: days_employed, dtype: float64

Можем видеть аномальные значения и безработных и пенсионеров.

Рассмотрим уникальные значения столбца `children`.

In [10]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

Снова наблюдаем аномальные значения, удаляем строки с ними.

In [11]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [12]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Изменение типов данных

Заменим вещественный тип данных в столбце `total_income` на целочисленный

In [15]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

In [30]:
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [16]:
data['education'] = data['education'].str.lower()

Также рассмотрим оставшиеся дубликаты и удалим их.

In [17]:
data.duplicated().sum()

np.int64(71)

In [18]:
data = data.drop_duplicates()

### Категоризация данных

Вручную проведем категоризацию данных по данной классификации:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


In [32]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [20]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

Также осуществим классификацию по уникальным целям взятия кредита.

In [21]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [35]:
data['purpose'].value_counts()

purpose
свадьба                                   790
на проведение свадьбы                     763
сыграть свадьбу                           760
операции с недвижимостью                  672
покупка коммерческой недвижимости         658
покупка жилья для сдачи                   649
операции с жильем                         647
операции с коммерческой недвижимостью     645
жилье                                     641
покупка жилья                             640
покупка жилья для семьи                   637
недвижимость                              631
строительство собственной недвижимости    628
операции со своей недвижимостью           623
строительство жилой недвижимости          620
строительство недвижимости                619
покупка своего жилья                      619
покупка недвижимости                      616
ремонт жилью                              604
покупка жилой недвижимости                602
на покупку своего автомобиля              504
заняться высшим образовани

В целом, можно выделить 4 основные категории взятия кредита:


- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

На их основе сформируем соответствующие категории.



In [ ]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [23]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

## Исследование данных

###  Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [24]:
data_pivot_children = data.pivot_table(index='children', values='debt', aggfunc=['sum', 'count', 'mean'])

data_pivot_children = data_pivot_children.droplevel(1,axis=1)\
.rename(columns={'sum':'Количество должников', 'count':'Всего кредитополучателей', 'mean':'Доля должников'})

data_pivot_children

,Количество должников,Всего кредитополучателей,Доля должников
children,,,
0,1063,14091,0.075438
1,444,4808,0.092346
2,194,2052,0.094542
3,27,330,0.081818
4,4,41,0.097561
5,0,9,0.000000


**Вывод:** 
у тех заемщиков, у которых отсутствуют дети, наблюдается пониженная задолженность в сравнении с теми, у кого дети есть. Наличие одного, двух и четырех детей градационно увеличивают возможность задолженности. 
Тем не менее, шанс задолженности у лиц с тремя детьми значительно ниже, чем у предыщущих категорий заемщиков с детьми, но немного выше, чем бездетных заемщиков. По категории лиц с пятью детьми недостаточно данных для формулирования корректных выводов.

 



### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [25]:
data_pivot_fam_status = data.pivot_table(index='family_status', values='debt', aggfunc=['sum', 'count', 'mean'])

data_pivot_fam_status = data_pivot_fam_status.droplevel(1,axis=1).\
rename(columns={'sum':'Количество должников', 'count':'Всего кредитополучателей', 'mean':'Доля должников'})

data_pivot_fam_status

,Количество должников,Всего кредитополучателей,Доля должников
family_status,,,
Не женат / не замужем,273,2796,0.097639
в разводе,84,1189,0.070648
вдовец / вдова,63,951,0.066246
гражданский брак,385,4134,0.093130
женат / замужем,927,12261,0.075606


**Вывод:** пониженная кредитоспособность наблюдается у не находящихся в браке лиц и лиц, находящихся в гражданском браке. 
Наименьшая вероятность задолженности свойственна категориям вдов/вдовцов и тех, кто находится в разводе, возможно это связанно с определенной степенью сознательности. Люди, находящиеся в браке, занимают промежуточное положение.

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [26]:
data_pivot_income = data.pivot_table(index='total_income_category', values='debt', aggfunc=['sum', 'count', 'mean'])

data_pivot_income = data_pivot_income.droplevel(1,axis=1)\
.rename(columns={'sum':'Количество должников', 'count':'Всего кредитополучателей', 'mean':'Доля должников'})

data_pivot_income


,Количество должников,Всего кредитополучателей,Доля должников
total_income_category,,,
A,2,25,0.080000
B,354,5014,0.070602
C,1353,15921,0.084982
D,21,349,0.060172
E,2,22,0.090909


**Вывод:** можно сказать, что по большинству категорий недостаточно данных для формулирования корректных выводов, получается слишком большая статпогрешность. Тем не менее, выводы можно сделать по категориям B и C. C (заработок от 50.000 до 200.000 руб.) - является менее кредитоспособной категорией, а категория B (заработок от 200.000 до 1.000.000 руб.) является боле безопасной в вопросе выплаты кредита. Можно сделать вывод, об отсутствии четкой градации в связи между доходами и возвратом кредита по данной таблице, это связано с недостатком данных.

### Как разные цели кредита влияют на его возврат в срок?

In [27]:
data_pivot_purposes = data.pivot_table(index='purpose_category', values='debt', aggfunc=['sum', 'count', 'mean'])

data_pivot_purposes = data_pivot_purposes.droplevel(1,axis=1)\
.rename(columns={'sum':'Количество должников', 'count':'Всего кредитополучателей', 'mean':'Доля должников'})


data_pivot_purposes


,Количество должников,Всего кредитополучателей,Доля должников
purpose_category,,,
операции с автомобилем,400,4279,0.093480
операции с недвижимостью,780,10751,0.072551
получение образования,369,3988,0.092528
проведение свадьбы,183,2313,0.079118


**Вывод:** недвижимость и свадьбы являются наиболее безопасными целями взятия кредита, а такие цели, как "операции с автомобилем" и "получение образования" несут больший риск задолженности.

### Возможные причины появления пропусков в исходных данных.

*Ответ:* пропуски в данных могут быть вызваны как технологическими причинами(ошибках при копировании, считывани и записи данных машиной), так и по причине обычного человеческого фактора.

### Почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

*Ответ:* медианные значения, в отличие от среднеарифметических, способны более точно оценивать диверсифицированные данные, в которых какие-то элементы выделяются среди большинства.

## Общий вывод

При работе с данными была проведена предобработка: обработаны дубликаты, удалены пропуски и различные аномальные значения, некоторые данные были преобразованы в более подходящие типы данных. 
Также была введена дополнительная категоризация данных, добавлены новые столбцы, которые использовались в дальнеших исследованиях. 

Было проведено 4 исследования о связях задолженности с различным категориям - количеством детей, семейным положением, уровнем дохода и целью кредита.
Итак, на основе проанализированных данных, можно сказать, что идеальным заемщиков является бездетный вдовец/вдова с заработком от 200.000 до 1.000.000, взявший кредит на недвижимость. 